# Retrieve all the GO terms for gene from the Uniprot database based on the classes

In [36]:
import numpy as np
import pandas as pd
import string
import os
from collections import Counter
from collections import defaultdict
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RandomizedLasso
from sklearn.feature_selection import RFE, f_regression, SelectFromModel, RFECV, SelectKBest, chi2
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re
from bioservices import *
import collections
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv('..//bases/training_variants')
test = pd.read_csv('..//bases/test_variants')

In [3]:
# only use gene from train data -> contains the classes
all_genes = set(train.Gene)
print(len(all_genes))
print(all_genes)

264
{'SMAD2', 'KMT2C', 'PRDM1', 'ACVR1', 'CIC', 'RRAS2', 'TET1', 'RYBP', 'HLA-B', 'U2AF1', 'SF3B1', 'NFE2L2', 'CTLA4', 'GNAQ', 'BRD4', 'CEBPA', 'ERBB4', 'ETV6', 'FOXO1', 'IDH2', 'YAP1', 'ARID5B', 'MEF2B', 'APC', 'MYC', 'DNMT3A', 'SRC', 'RNF43', 'IKZF1', 'IL7R', 'PTEN', 'PTPRD', 'ARID1B', 'MSH2', 'GNA11', 'MGA', 'STK11', 'FOXP1', 'CDKN2B', 'FGF19', 'PIK3CD', 'FOXL2', 'MLH1', 'FGF3', 'ERG', 'MET', 'NSD1', 'IGF1R', 'NF1', 'RAB35', 'PPM1D', 'RBM10', 'PIK3CB', 'KMT2A', 'PPP2R1A', 'NOTCH1', 'TET2', 'BARD1', 'SMO', 'TSC1', 'RUNX1', 'IKBKE', 'TERT', 'CREBBP', 'FGFR2', 'MAP2K2', 'ERBB2', 'KDM6A', 'MAP2K4', 'KMT2B', 'MYD88', 'FUBP1', 'TP53', 'FGFR4', 'AXL', 'CCND2', 'FLT1', 'CDKN2A', 'SOS1', 'RHEB', 'FANCA', 'RAD21', 'CHEK2', 'TSC2', 'PIK3R3', 'XRCC2', 'RICTOR', 'ABL1', 'MTOR', 'MAP3K1', 'SHOC2', 'MDM2', 'FBXW7', 'KNSTRN', 'NOTCH2', 'GNAS', 'KRAS', 'DDR2', 'ERCC3', 'ETV1', 'LATS1', 'CDK8', 'PIM1', 'BCL10', 'CDK12', 'FLT3', 'EPAS1', 'AR', 'AURKA', 'KMT2D', 'MYCN', 'CARM1', 'GLI1', 'PDGFRA', 'SPOP

In [4]:
u = UniProt()

In [5]:
res = u.search("ZAP70_HUMAN")
print(res)

Entry	Entry name	Status	Protein names	Gene names	Organism	Length
P43403	ZAP70_HUMAN	reviewed	Tyrosine-protein kinase ZAP-70 (EC 2.7.10.2) (70 kDa zeta-chain associated protein) (Syk-related tyrosine kinase)	ZAP70 SRK	Homo sapiens (Human)	619



In [6]:
u.debugLevel = "INFO"
u.timeout = 100   # some queries are long and requires much more time; default is 1000 seconds

In [7]:
# just an example of query
a = u.search('SLC16A1+AND+organism:9606', frmt='tab', limit=1,
               columns="entry name")

In [8]:
[s.strip() for s in a.splitlines()]

['Entry name', 'MOT1_HUMAN']

In [9]:
gene_entry_dict = {} # here we will keep the gene_entries together with their classes
class_dict = {}
for gene in all_genes:
    gene_classes = list(train.Class[train.Gene==gene])
    keyword = 'gene:%s+AND+organism:9606' %gene #to query database, with gene and organism 9606 is Homo Sapien (human)
    entry_name_tab = u.search(keyword, frmt='tab', limit=1, columns="entry name") 
    entry_name = [s.strip() for s in entry_name_tab.splitlines()][1] # gets the entry name from uniprot i.e. second position in tab
    gene_entry_dict[gene] = entry_name
    class_dict[entry_name] = gene_classes

In [10]:
gene_entry_dict

{'ABL1': 'ABL1_HUMAN',
 'ACVR1': 'ACVR1_HUMAN',
 'AGO2': 'AGO2_HUMAN',
 'AKT1': 'AKT1_HUMAN',
 'AKT2': 'AKT2_HUMAN',
 'AKT3': 'AKT3_HUMAN',
 'ALK': 'TGFR1_HUMAN',
 'APC': 'APC_HUMAN',
 'AR': 'ANDR_HUMAN',
 'ARAF': 'ARAF_HUMAN',
 'ARID1A': 'ARI1A_HUMAN',
 'ARID1B': 'ARI1B_HUMAN',
 'ARID2': 'ARID2_HUMAN',
 'ARID5B': 'ARI5B_HUMAN',
 'ASXL1': 'ASXL1_HUMAN',
 'ASXL2': 'ASXL2_HUMAN',
 'ATM': 'ATM_HUMAN',
 'ATR': 'ATR_HUMAN',
 'ATRX': 'ATRX_HUMAN',
 'AURKA': 'AURKA_HUMAN',
 'AURKB': 'AURKB_HUMAN',
 'AXIN1': 'AXIN1_HUMAN',
 'AXL': 'UFO_HUMAN',
 'B2M': 'B2MG_HUMAN',
 'BAP1': 'RING2_HUMAN',
 'BARD1': 'BARD1_HUMAN',
 'BCL10': 'BCL10_HUMAN',
 'BCL2': 'BCL2_HUMAN',
 'BCL2L11': 'B2L11_HUMAN',
 'BCOR': 'BCOR_HUMAN',
 'BRAF': 'BRAF_HUMAN',
 'BRCA1': 'BRCA1_HUMAN',
 'BRCA2': 'BRCA2_HUMAN',
 'BRD4': 'BRD4_HUMAN',
 'BRIP1': 'FANCJ_HUMAN',
 'BTK': 'BTK_HUMAN',
 'CARD11': 'CAR11_HUMAN',
 'CARM1': 'CARM1_HUMAN',
 'CASP8': 'CASP8_HUMAN',
 'CBL': 'CBL_HUMAN',
 'CCND1': 'CCND1_HUMAN',
 'CCND2': 'CCND2_HUMAN',


In [12]:
gene_entries = list(gene_entry_dict.values())
len(gene_entries)

264

In [13]:
df = u.get_df(gene_entries)
df

INFO:root:fetching information from uniprot for 262 entries
INFO:root:uniprot.get_df 1/2
INFO:root:uniprot.get_df 2/2
INFO:root:uniprot.get_df 3/2


,Entry,Entry name,Gene names,Gene names (primary ),Gene names (synonym ),Gene names (ordered locus ),Gene names (ORF ),Organism,Organism ID,Protein names,...,Miscellaneous [CC],Keywords,Protein existence,Status,Sequence annotation (Features),Protein families,Version,Comments,Cross-reference (null),Pathway.1
0,P35222,CTNB1_HUMAN,[CTNNB1 CTNNB OK/SW-cl.35 PRO2286],CTNNB1,CTNNB,NaN,OK/SW-cl.35 PRO2286,Homo sapiens (Human),9606,Catenin beta-1 (Beta-catenin),...,NaN,"[3D-structure, Acetylation, Activator, Alterna...",Evidence at protein level,reviewed,NaN,[Beta-catenin family],216,"[Alternative products (1), Caution (1), Functi...",NaN,NaN
1,Q12809,KCNH2_HUMAN,[KCNH2 ERG ERG1 HERG],KCNH2,ERG ERG1 HERG,NaN,NaN,Homo sapiens (Human),9606,Potassium voltage-gated channel subfamily H me...,...,NaN,"[3D-structure, Alternative splicing, Cell memb...",Evidence at protein level,reviewed,NaN,"[Potassium channel family, H (Eag) (TC 1.A.1.2...",199,"[Alternative products (1), Caution (3), Domain...",NaN,NaN
2,P35968,VGFR2_HUMAN,[KDR FLK1 VEGFR2],KDR,FLK1 VEGFR2,NaN,NaN,Homo sapiens (Human),9606,Vascular endothelial growth factor receptor 2 ...,...,NaN,"[3D-structure, ATP-binding, Alternative splici...",Evidence at protein level,reviewed,NaN,"[Protein kinase superfamily, Tyr protein kinas...",200,"[Alternative products (1), Catalytic activity ...",NaN,NaN
3,P42336,PK3CA_HUMAN,[PIK3CA],PIK3CA,NaN,NaN,NaN,Homo sapiens (Human),9606,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",...,MISCELLANEOUS: The avian sarcoma virus 16 geno...,"[3D-structure, ATP-binding, Angiogenesis, Comp...",Evidence at protein level,reviewed,NaN,[PI3/PI4-kinase family],189,"[Catalytic activity (2), Domain (1), Function ...",NaN,NaN
4,Q09472,EP300_HUMAN,[EP300 P300],EP300,P300,NaN,NaN,Homo sapiens (Human),9606,Histone acetyltransferase p300 (p300 HAT) (EC ...,...,NaN,"[3D-structure, Acetylation, Acyltransferase, B...",Evidence at protein level,reviewed,NaN,[],222,"[Catalytic activity (1), Domain (1), Function ...",NaN,NaN
5,P42771,CDN2A_HUMAN,[CDKN2A CDKN2 MTS1],CDKN2A,CDKN2 MTS1,NaN,NaN,Homo sapiens (Human),9606,Cyclin-dependent kinase inhibitor 2A (Cyclin-d...,...,NaN,"[3D-structure, ANK repeat, Acetylation, Altern...",Evidence at protein level,reviewed,NaN,[CDKN2 cyclin-dependent kinase inhibitor family],197,"[Alternative products (1), Caution (2), Functi...",NaN,NaN
6,P12830,CADH1_HUMAN,[CDH1 CDHE UVO],CDH1,CDHE UVO,NaN,NaN,Homo sapiens (Human),9606,Cadherin-1 (CAM 120/80) (Epithelial cadherin) ...,...,NaN,"[3D-structure, Alternative splicing, Calcium, ...",Evidence at protein level,reviewed,NaN,[],219,"[Alternative products (1), Domain (1), Functio...",NaN,NaN
7,P84022,SMAD3_HUMAN,[SMAD3 MADH3],SMAD3,MADH3,NaN,NaN,Homo sapiens (Human),9606,Mothers against decapentaplegic homolog 3 (MAD...,...,NaN,"[3D-structure, ADP-ribosylation, Acetylation, ...",Evidence at protein level,reviewed,NaN,[Dwarfin/SMAD family],163,"[Alternative products (1), Caution (2), Domain...",NaN,NaN
8,P40692,MLH1_HUMAN,[MLH1 COCA2],MLH1,COCA2,NaN,NaN,Homo sapiens (Human),9606,DNA mismatch repair protein Mlh1 (MutL protein...,...,NaN,"[3D-structure, ATP-binding, Acetylation, Alter...",Evidence at protein level,reviewed,NaN,[DNA mismatch repair MutL/HexB family],200,"[Alternative products (1), Caution (1), Functi...",NaN,NaN
9,P03372,ESR1_HUMAN,[ESR1 ESR NR3A1],ESR1,ESR NR3A1,NaN,NaN,Homo sapiens (Human),9606,Estrogen receptor (ER) (ER-alpha) (Estradiol r...,...,MISCELLANEOUS: Selective estrogen receptor mod...,"[3D-structure, Activator, Alternative promoter...",Evidence at protein level,reviewed,NaN,"[Nuclear hormone receptor family, NR3 subfamily]",243,"[Alternative products (1), Caution (3), Domain...",NaN,NaN


In [14]:
df['Gene ontology (GO)'][0]

['adherens junction [GO:0005912]',
 'apical part of cell [GO:0045177]',
 'basolateral plasma membrane [GO:0016323]',
 'beta-catenin destruction complex [GO:0030877]',
 'beta-catenin-TCF complex [GO:1990907]',
 'beta-catenin-TCF7L2 complex [GO:0070369]',
 'bicellular tight junction [GO:0005923]',
 'catenin complex [GO:0016342]',
 'cell cortex [GO:0005938]',
 'cell junction [GO:0030054]',
 'cell periphery [GO:0071944]',
 'cell-cell adherens junction [GO:0005913]',
 'cell-cell junction [GO:0005911]',
 'centrosome [GO:0005813]',
 'cytoplasm [GO:0005737]',
 'cytosol [GO:0005829]',
 'extracellular exosome [GO:0070062]',
 'fascia adherens [GO:0005916]',
 'flotillin complex [GO:0016600]',
 'focal adhesion [GO:0005925]',
 'intracellular [GO:0005622]',
 'lamellipodium [GO:0030027]',
 'lateral plasma membrane [GO:0016328]',
 'membrane [GO:0016020]',
 'microvillus membrane [GO:0031528]',
 'nuclear euchromatin [GO:0005719]',
 'nucleoplasm [GO:0005654]',
 'nucleus [GO:0005634]',
 'perinuclear region

In [15]:
GO_terms_dict = dict(zip(df['Entry name'], df['Gene ontology (GO)']))

In [16]:
GO_terms_dict

{'1A02_HUMAN': ['cell surface [GO:0009986]',
  'early endosome membrane [GO:0031901]',
  'endoplasmic reticulum [GO:0005783]',
  'endoplasmic reticulum exit site [GO:0070971]',
  'endoplasmic reticulum membrane [GO:0005789]',
  'ER to Golgi transport vesicle membrane [GO:0012507]',
  'Golgi apparatus [GO:0005794]',
  'Golgi medial cisterna [GO:0005797]',
  'Golgi membrane [GO:0000139]',
  'integral component of lumenal side of endoplasmic reticulum membrane [GO:0071556]',
  'MHC class I protein complex [GO:0042612]',
  'phagocytic vesicle membrane [GO:0030670]',
  'plasma membrane [GO:0005886]',
  'recycling endosome membrane [GO:0055038]',
  'beta-2-microglobulin binding [GO:0030881]',
  'peptide antigen binding [GO:0042605]',
  'receptor binding [GO:0005102]',
  'RNA binding [GO:0003723]',
  'T cell receptor binding [GO:0042608]',
  'TAP binding [GO:0046977]',
  'antibacterial humoral response [GO:0019731]',
  'antigen processing and presentation of endogenous peptide antigen via MHC

In [17]:
# Find most common GO terms to use as features
def flatten(l): # taken from https://stackoverflow.com/questions/33900770/most-frequent-values-in-a-dictionary
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, str): #replaced basestring with str for Python3
            for sub in flatten(el):
                yield sub
        else:
            yield el



In [18]:
All_GO_terms = list(set((flatten(GO_terms_dict.values())))) # we want list of the unique values (set) to use for modelling
len(All_GO_terms)

3802

In [19]:
# Initialization of the 3802 new features with 0's
for terms in All_GO_terms:
    train[terms] = 0

In [25]:
# looping through all classes and getting terms for each class
'''terms_per_class = defaultdict(list)
for entry, terms in GO_terms_dict.items():
    if entry in class_dict:
        gene_classes = class_dict[entry]
        for gene_class in gene_classes:
            terms_per_class[gene_class].extend(terms)
           
        
terms_per_class'''

'terms_per_class = defaultdict(list)\nfor entry, terms in GO_terms_dict.items():\n    if entry in class_dict:\n        gene_classes = class_dict[entry]\n        for gene_class in gene_classes:\n            terms_per_class[gene_class].extend(terms)\n           \n        \nterms_per_class'

In [24]:
# code if we want most commons
'''counter_dict = {}
for classes in terms_per_class:
    counter_dict[classes] = Counter(terms_per_class[classes]).most_common(50)'''


'counter_dict = {}\nfor classes in terms_per_class:\n    counter_dict[classes] = Counter(terms_per_class[classes]).most_common(50)'

In [20]:
# adds the molecular function GO terms to each gene in train data frame
for i in train.index:
    gene = train.Gene[i]
    gene_entry = gene_entry_dict[gene]
    if gene_entry in GO_terms_dict:
        GO_terms = GO_terms_dict[gene_entry]
        train.loc[i, GO_terms] = 1

train.shape

(3321, 3806)

In [21]:
train

,ID,Gene,Variation,Class,cardiac right atrium morphogenesis [GO:0003213],type 1 angiotensin receptor binding [GO:0031702],negative regulation of triglyceride biosynthetic process [GO:0010868],collecting duct development [GO:0072044],embryonic skeletal limb joint morphogenesis [GO:0036023],response to starvation [GO:0042594],...,catalytic step 2 spliceosome [GO:0071013],response to molecule of bacterial origin [GO:0002237],sulfur compound metabolic process [GO:0006790],defense response to virus [GO:0051607],evasion or tolerance of host defenses by virus [GO:0019049],cellular response to lipopolysaccharide [GO:0071222],protein C-terminus binding [GO:0008022],DNA-methyltransferase activity [GO:0009008],regulation of cell-cell adhesion [GO:0022407],VEGF-activated neuropilin signaling pathway [GO:0038190]
0,0,FAM58A,Truncating Mutations,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,CBL,W802*,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,CBL,Q249E,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,CBL,N454D,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,CBL,L399V,4,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,5,CBL,V391I,4,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,6,CBL,V430M,5,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,7,CBL,Deletion,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,8,CBL,Y371H,4,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,9,CBL,C384R,4,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [90]:
train.to_csv("new_train_uniprot.csv",index=False)

In [22]:
# fit the input X and output Y for the feature selection
X = train[All_GO_terms]
y = train['Class']
names = X.columns
ranks = {}

In [24]:
def rank_to_dict(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x, 2), ranks)
    return dict(zip(names, ranks ))

In [71]:
# Lasso model
lasso = Lasso(alpha=.002, random_state = 3).fit(X,y)
features_lasso = names[np.nonzero(lasso.coef_)]
features_lasso # 182 in total

Index(['negative regulation of glial cell proliferation [GO:0060253]',
       'cellular response to UV [GO:0034644]',
       'activation of protein kinase activity [GO:0032147]',
       'positive regulation of neuron apoptotic process [GO:0043525]',
       'positive regulation of endothelial cell proliferation [GO:0001938]',
       'regulation of interferon-gamma-mediated signaling pathway [GO:0060334]',
       'ATP binding [GO:0005524]', 'RISC-loading complex [GO:0070578]',
       'miRNA metabolic process [GO:0010586]',
       'bone mineralization [GO:0030282]',
       ...
       'B cell proliferation [GO:0042100]',
       'RNA polymerase II core promoter sequence-specific DNA binding [GO:0000979]',
       'methylcytosine dioxygenase activity [GO:0070579]',
       'double-strand break repair [GO:0006302]',
       'phosphotyrosine residue binding [GO:0001784]',
       'scaffold protein binding [GO:0097110]',
       'glyoxylate cycle [GO:0006097]', 'response to estradiol [GO:0032355]',


In [84]:
# L1-SVC model
lsvc = LinearSVC(C=0.02, penalty="l1", dual=False, random_state = 3).fit(X, y)
features_lsvc = names[np.nonzero(lsvc.coef_)[1]]
features_lsvc # 209 in total

Index(['embryonic organ development [GO:0048568]', 'ATP binding [GO:0005524]',
       'positive regulation of ERK1 and ERK2 cascade [GO:0070374]',
       'cytosol [GO:0005829]',
       'protein serine/threonine kinase activity [GO:0004674]',
       'plasma membrane [GO:0005886]', 'DNA binding [GO:0003677]',
       'protein complex assembly [GO:0006461]',
       'chromosome segregation [GO:0007059]',
       'positive regulation of gene expression [GO:0010628]',
       ...
       'cell cycle arrest [GO:0007050]', 'nucleus [GO:0005634]',
       'thymus development [GO:0048538]', 'plasma membrane [GO:0005886]',
       'ATP binding [GO:0005524]', 'cytosol [GO:0005829]',
       'plasma membrane [GO:0005886]',
       'positive regulation of transcription from RNA polymerase II promoter [GO:0045944]',
       'positive regulation of transcription, DNA-templated [GO:0045893]',
       'catalytic step 2 spliceosome [GO:0071013]'],
      dtype='object', length=209)

In [89]:
features_lsvc.intersection(features_lasso)

Index(['ATP binding [GO:0005524]', 'cytosol [GO:0005829]',
       'protein serine/threonine kinase activity [GO:0004674]',
       'plasma membrane [GO:0005886]', 'DNA binding [GO:0003677]',
       'chromosome segregation [GO:0007059]',
       'positive regulation of gene expression [GO:0010628]',
       'MAPK cascade [GO:0000165]',
       'regulation of cell proliferation [GO:0042127]',
       'viral process [GO:0016032]', 'beta-catenin binding [GO:0008013]',
       'covalent chromatin modification [GO:0016569]',
       'signal transduction [GO:0007165]',
       'G1/S transition of mitotic cell cycle [GO:0000082]',
       'protein complex [GO:0043234]', 'cell surface [GO:0009986]',
       'plasma membrane [GO:0005886]', 'apoptotic process [GO:0006915]',
       'nucleoplasm [GO:0005654]', 'signal transduction [GO:0007165]',
       'protein homodimerization activity [GO:0042803]',
       'protein deubiquitination [GO:0016579]', 'protein complex [GO:0043234]',
       'somatic stem cell po

In [52]:
forest = ExtraTreesClassifier(n_estimators=200,
                              random_state=6)
forest.fit(X, y)
model = SelectFromModel(forest, prefit=True)
X_new = model.transform(X)
X_new.shape # reduced to 822 features

(3321, 821)

In [113]:
feature_index

[2121,
 3550,
 1277,
 2158,
 2362,
 3741,
 1760,
 225,
 2516,
 2335,
 1346,
 2205,
 3426,
 3216,
 2695,
 3653,
 33,
 738,
 641,
 2152,
 3428,
 2847,
 1913,
 2295,
 714,
 1542,
 3068,
 1884,
 455,
 1762,
 479,
 266,
 3060,
 1846,
 805,
 283,
 477,
 2058,
 3143,
 2928,
 1128,
 644,
 1828,
 3196,
 832,
 1740,
 1188,
 3386,
 2654,
 861,
 888,
 2313,
 3571,
 873,
 2970,
 2542,
 3552,
 2740,
 3463,
 2765,
 3245,
 3532,
 3421,
 8,
 1989,
 2061,
 1413,
 2311,
 51,
 1799,
 2296,
 1305,
 509,
 6,
 808,
 2864,
 825,
 2736,
 1727,
 677,
 510,
 3044,
 2199,
 914,
 904,
 2150,
 3070,
 1794,
 3742,
 2920,
 2167,
 1670,
 2129,
 2997,
 922,
 175,
 3471,
 2370,
 2325,
 3491,
 335,
 717,
 233,
 3684,
 1143,
 1917,
 10,
 1026,
 3199,
 1547,
 3582,
 1235,
 1202,
 3617,
 630,
 3304,
 2072,
 1920,
 1001,
 2707,
 3165,
 1244,
 524,
 2553,
 900,
 3750,
 2020,
 342,
 1096,
 676,
 1414,
 907,
 2317,
 1134,
 3195,
 161,
 3052,
 1738,
 1079,
 1127,
 2835,
 2218,
 1174,
 3134,
 913,
 1054,
 3272,
 417,
 1891,
 3002

In [123]:
train[train.ix[:,1760]==1]

,ID,Gene,Variation,Class,cell-cell junction [GO:0005911],cis-Golgi network [GO:0005801],cytoplasm [GO:0005737],cytosol [GO:0005829],membrane [GO:0016020],nucleus [GO:0005634],...,nuclear export signal receptor activity [GO:0005049],nucleocytoplasmic transporter activity [GO:0005487],Ran GTPase binding [GO:0008536],transporter activity [GO:0005215],mRNA transport [GO:0051028],protein export from nucleus [GO:0006611],regulation of centrosome duplication [GO:0010824],ribosomal large subunit export from nucleus [GO:0000055],ribosomal small subunit export from nucleus [GO:0000056],ribosomal subunit export from nucleus [GO:0000054]
2960,2960,KIT,D816E,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2961,2961,KIT,K558N,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2962,2962,KIT,L576del,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2963,2963,KIT,D816F,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2964,2964,KIT,A829P,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2965,2965,KIT,D816N,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2966,2966,KIT,I563_L576del,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2967,2967,KIT,T670I,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2968,2968,KIT,E839K,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2969,2969,KIT,K642E,7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# saving the train set together with all features from uniprot
train.to_csv("train_uniprot.csv",index=False)

In [58]:
# loading the XGboost most important 190 features
feature_scores = np.load("features_ranking.npy")

In [59]:
features = []
for feature_score in feature_scores:
    feature = feature_score[0]
    features.append(feature)

In [66]:
features

['magnesium ion binding [GO:0000287]',
 'protein tyrosine kinase activity [GO:0004713]',
 'enzyme binding [GO:0019899]',
 'ATP binding [GO:0005524]',
 'kinase activity [GO:0016301]',
 'transcription corepressor activity [GO:0003714]',
 'protein C-terminus binding [GO:0008022]',
 'H3 histone acetyltransferase activity [GO:0010484]',
 'damaged DNA binding [GO:0003684]',
 'protein kinase activity [GO:0004672]',
 'zinc ion binding [GO:0008270]',
 'ATPase activity [GO:0016887]',
 'tubulin binding [GO:0015631]',
 'protein homodimerization activity [GO:0042803]',
 'protein heterodimerization activity [GO:0046982]',
 'protein kinase binding [GO:0019901]',
 'protein serine/threonine kinase activity [GO:0004674]',
 'ubiquitin-protein transferase activity [GO:0004842]',
 'identical protein binding [GO:0042802]',
 'protein complex binding [GO:0032403]',
 'GTPase activity [GO:0003924]',
 'RNA polymerase II core promoter sequence-specific DNA binding [GO:0000979]',
 'transmembrane receptor protein t

In [91]:
# adding only the 190 most important features from XGboost + the dummy variables of gene
train_features = train[features]
train_original = pd.read_csv('..//bases/training_variants')
train_dummy = pd.get_dummies(train_original.Gene) 
train_new = pd.concat([train_original, train_features, train_dummy], axis=1)
train_new.shape

(3321, 458)

In [94]:
train_new


,ID,Gene,Variation,Class,magnesium ion binding [GO:0000287],protein tyrosine kinase activity [GO:0004713],enzyme binding [GO:0019899],ATP binding [GO:0005524],kinase activity [GO:0016301],transcription corepressor activity [GO:0003714],...,TSC1,TSC2,U2AF1,VEGFA,VHL,WHSC1,WHSC1L1,XPO1,XRCC2,YAP1
0,0,FAM58A,Truncating Mutations,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,CBL,W802*,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,CBL,Q249E,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,CBL,N454D,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,CBL,L399V,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,CBL,V391I,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,CBL,V430M,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,CBL,Deletion,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,CBL,Y371H,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,CBL,C384R,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# save train_new somewhere
